In [12]:
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import xgboost
from xgboost import XGBClassifier

In [13]:
folder_path = r'C:\Users\Tim Chen\OneDrive\桌面\台大\台大機器學習\html.2023.final.data\release'
rain_path = 'C:/Users/Tim Chen/OneDrive/桌面/台大/台大機器學習/機器學習專案/extend_X_feature/weather/If_rain.xlsx'
holiday_path = 'C:/Users/Tim Chen/OneDrive/桌面/台大/台大機器學習/機器學習專案/extend_X_feature/holiday/If_holiday.xlsx'
all_the_stops_path = 'C:/Users/Tim Chen/OneDrive/桌面/台大/台大機器學習/html.2023.final.data/demographic.json'

In [14]:
with open(all_the_stops_path, 'r', encoding='utf-8') as json_file:
    parsed_data = json.load(json_file)
    all_the_stops = list(parsed_data.keys())

In [15]:
def read_feature(feature_path, feature_name):#讀取feature資料，資料我自己編製的excel檔案
    df = pd.read_excel(feature_path)
    df_dict = {}
    for index, row in df.iterrows():
        key = int(row['date'])
        value = row[feature_name]
        df_dict[str(key)] = value
    return df_dict

In [16]:
def read_json_files(folder_path, specific_bike_stop, rain_data, holiday_data):#讀老師給的檔案，怎麼讀的不太重要，因為要看資料的邏輯，所以知道是讀資料就好
    Date = []#注意參數stop_point我指的是讀幾個車站的意思，你要輸出五個車站就打五
    Data = []#rain_data跟holiday_data都是我目前先用很硬幹的方法輸入這個function，你可以參考我的feature然後做一個feature的excel檔案給我，我明天讀

    
    for root, dirs, files in os.walk(folder_path):
        
        if str(root)[-8:][0] != '2':
            continue
        
        for file in files:
            #print(str(file))
            if file.endswith('.json') and str(file)[:9] == specific_bike_stop:
                #print(f'{str(root)[-8:]} :yes')
                file_path = os.path.join(root, file)
  
                with open(file_path, 'r', encoding='utf-8') as json_file:
                    data = json.load(json_file)
                    
                    for time, value in data.items():
                        if value:
                            entry = {
                                'date': str(root)[-8:],
                                'bike_stop': str(file)[:9],
                                'time': time,
                                'total': value["tot"],
                                'current': value["sbi"],
                                'can_park': value["bemp"],
                                'open': value["act"],
                                'rain_hour': rain_data[f'{str(root)[-8:]}'],
                                'holiday': holiday_data[f'{str(root)[-8:]}']
                            }
                            Data.append(entry)
        Date.append({str(root)[-8:]})
    
    return Data, Date, 

In [17]:
def time_to_minutes(time_str):#時間是str不能丟train，我改成用總分鐘數，他就會遞增然後是int
    hour, minute = map(int, time_str.split(':'))
    return hour * 60 + minute

In [18]:
def err(y_test, y_pred, total_stop):#老師講義的error function，照打而已
    sum = 0
    for i in range(y_test.size):
        yt = y_test[i]
        yp = y_pred[i]
        sum += 3*(abs(yt-yp)/total_stop)*(abs(yt/total_stop-1/3)+abs(yt/total_stop-2/3))
    sum /= y_test.size
    return sum

In [19]:
best_models = {}

In [20]:
def trainModels(new_data, best_models):
    for stop in new_data:#這裡stop指的是車站名稱
        print("Bike stop ID: ", stop)
        X = []
        y = []
        total_stop = 0
        for elem in new_data[stop]:#這裡開始把資料轉為Xy形式，簡單來說，讓機器看得懂，理論上一個車站X.shape -> (very big, 5), 前面這個5是feature的數量， y.shape -> (very big), 這very big值相同 主要跟你讀進幾個車站的資料成正比
            total_stop = elem.get('total')
            if elem.get('open') == 0:
                continue
            X.append([time_to_minutes(elem.get('time')), elem.get('total'), elem.get('open'), elem.get('rain_hour'), elem.get('holiday')])
            y.append(elem.get('current'))
        X = np.array(X)
        #print(X.shape)
        y = np.array(y)
        X_numeric = X.astype(float)
        smallest_error = float('inf')
        best_model = XGBClassifier(n_estimators=100, max_depth = 10, random_state=42) #initialize 隨機森林的model
        for i in range(1):#我做十次，用不同的資料切割跑，使用最小的error的那個
            X_train, X_val, y_train, y_val = train_test_split(X_numeric, y, test_size=0.2, random_state=42)#資料的切割，這公式很好用 0.1是指切多少當validation，每跑一次用跟前一次不同的亂數種，注意亂數種不用每次compile都不同，因為想比較演算法能不能在同資料下變更好
            
            model = XGBClassifier(n_estimators= 200, learning_rate= 0.1)
            model.fit(X_train, y_train)
            
            y_pred = model.predict(X_val)
            #print(y_test)
            #print(y_pred)  
            #for true_value, predicted_value in zip(y_test, y_pred):
                #print(f"True: {true_value}, Predicted: {predicted_value}")
            #print("正確率", model.score(X_val,y_val))
            error = err(y_val, y_pred, total_stop)
            if error < smallest_error:
                smallest_error = error
                best_model = model
                
        best_models[str(stop)] = best_model
        print(smallest_error)



In [21]:
def transformData(data, specific_bike_stop):
    new_data = {}
    new_data[specific_bike_stop] = []

    for elem in data:#原先資料讀進來是某日->車站->data, 我改成車站->某日->data。簡單來說，是要讓每個車站都有一個model，資料要長這樣比較好讀
        bike_stop = elem.get('bike_stop')
        if bike_stop == specific_bike_stop:
            new_data[bike_stop].append(elem)
    return new_data

In [22]:
rain_data = read_feature(rain_path, 'rain_hour')
holiday_data = read_feature(holiday_path, "holiday")
set_stop = 0
for specific_bike_stop in all_the_stops:
    if set_stop == 10:
        break
    data, date = read_json_files(folder_path, specific_bike_stop, rain_data, holiday_data)
    new_data = transformData(data, specific_bike_stop)
    trainModels(new_data, best_models)
    set_stop += 1

Bike stop ID:  500101001
0.1650014951035349
Bike stop ID:  500101002


KeyboardInterrupt: 

In [ ]:
def find_total_stops(specific_bike_stop):
    data, date = read_json_files(folder_path, specific_bike_stop, rain_data, holiday_data)
    new_data = transformData(data, specific_bike_stop)
    return new_data[specific_bike_stop][1]['total']

In [ ]:
import csv
predict_path = 'C:/Users/Tim Chen/OneDrive/桌面/台大/台大機器學習/html.2023.final.data/sample_submission_stage1.csv'
df = pd.read_csv(predict_path)
predict_time = df['id'].tolist()

weather_forecast_path = 'C:/Users/Tim Chen/OneDrive/桌面/台大/台大機器學習/機器學習專案/extend_X_feature/predict_date_rain_hour/predict_rain_hour.xlsx'
pred_rain = read_feature(weather_forecast_path, "rain_hour")
the_holiday = read_feature(weather_forecast_path, "holiday")

my_prediction = []
timer = 0

for elem in predict_time:
    timer += 1
    x_to_pred = []
    future_date = elem[:8]
    stop_to_predict = elem[9:18]
    time_to_predict = time_to_minutes(elem[19:])
        
    rain_hour = pred_rain[future_date]
    holiday = the_holiday[future_date]
    
    x_to_pred.append([time_to_predict, find_total_stops(stop_to_predict), 1, rain_hour, holiday])
    y_pred = best_models[stop_to_predict].predict(x_to_pred)
    my_prediction.append(y_pred)
    
    if timer == 72:
        timer = 0
        print(f'{stop_to_predict}, {future_date}')

500101001, 20231204
500101002, 20231204
500101003, 20231204
500101004, 20231204
500101005, 20231204
500101006, 20231204
500101007, 20231204
500101008, 20231204
500101009, 20231204
500101010, 20231204
500101013, 20231204
500101014, 20231204
500101015, 20231204
500101018, 20231204
500101019, 20231204
500101020, 20231204
500101021, 20231204
500101022, 20231204
500101023, 20231204
500101024, 20231204
500101025, 20231204
500101026, 20231204
500101027, 20231204
500101028, 20231204
500101029, 20231204
500101030, 20231204
500101031, 20231204
500101032, 20231204
500101033, 20231204
500101034, 20231204
500101035, 20231204
500101036, 20231204
500101037, 20231204
500101038, 20231204
500101039, 20231204
500101040, 20231204
500101041, 20231204
500101042, 20231204
500101091, 20231204
500101092, 20231204
500101093, 20231204
500101094, 20231204
500101114, 20231204
500101115, 20231204
500101123, 20231204
500101166, 20231204
500101175, 20231204
500101176, 20231204
500101181, 20231204
500101184, 20231204


In [ ]:
import joblib
def find_key(dictionary, value):
    for key, val in dictionary.items():
        if val == value:
            return key
    return None

In [ ]:
import os
import joblib

# Create a folder to store the models if it doesn't exist
custom_path = r"C:\Users\Tim Chen\OneDrive\桌面\台大\台大機器學習\機器學習專案\models"
os.makedirs(custom_path, exist_ok=True)

# Iterate through the models and store them in the folder
for key, model in best_models.items():
    model_filename = os.path.join(custom_path, f'model_{str(key)}.pkl')
    joblib.dump(model, model_filename)

In [ ]:
print(my_prediction)

[18, 17, 13, 12, 13, 13, 13, 13, 13, 13, 13, 18, 18, 18, 18, 18, 17, 17, 18, 22, 19, 13, 6, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 2, 3, 6, 5, 7, 8, 10, 27, 27, 27, 13, 19, 12, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 18, 19, 8, 7, 7, 7, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 8, 8, 8, 9, 9, 11, 11, 11, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 3, 6, 2, 7, 21, 21, 21, 21, 19, 12, 7, 6, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 2, 1, 1, 1, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 5, 7, 7, 7, 7, 5, 7, 7, 6, 7, 7, 8, 5, 5, 5, 6, 6, 6, 6, 8, 11, 11, 11, 11, 11, 11, 11, 13, 12, 12, 12, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 11, 11, 1, 0, 0, 0, 0, 0, 0, 0, 3, 4, 4, 7, 10, 11, 11, 11, 11, 11, 11, 11, 6, 6, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0, 1, 3, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 6, 10, 2

In [ ]:
import openpyxl
# Specify the Excel file name to save
excel_filename = "my_prediction.xlsx"

# Create a new Excel workbook
wb = openpyxl.Workbook()

# Select the active sheet
ws = wb.active

# Write the 1D array data to a single column in the Excel sheet
for item in my_prediction:
    ws.append([item])

# Save the Excel workbook to a file
directory_path = r"C:\Users\Tim Chen\OneDrive\桌面\台大\台大機器學習\機器學習專案"

excel_filepath = os.path.join(directory_path, excel_filename)
wb.save(excel_filepath)